# RSI線交易策略驗證
Author：余慶龍  
Establish Date：2021.7.25  
Last Modified Date：2021.9.10

## Table of Contents
1. 讀取檔案模塊
2. 指標與交易策略
3. 回測模塊
4. 執行回測並搜集資料
5. 統計結果

## 1. 讀取檔案模塊

In [2]:
"""
把csv或excel檔案轉換成dataframe
以供後續的pandas操作
"""

import pandas as pd
import numpy as np
from datetime import datetime

def readStock_file(file, filetype='csv'):


    if filetype == 'excel':
        df = pd.read_excel(file, engine='openpyxl', parse_dates=True, header=None)
    else:
        df = pd.read_csv(file)

    # 取代原本的 column 名稱
    # 檔案的日期與開高低收需要照這個順序
    colume_name = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.columns = colume_name

    # 用日期這一行當做 df 的索引
    df = df.set_index('Date')

    # 把日期轉成 datetime的格式(從string)
    df.index = pd.to_datetime(df.index)

    # 照日期排序並把空資料轉成numpy的nan
    df = df.sort_index()
    df = df.replace(r'^\s*-$', np.nan, regex=True)


    for col in df.columns:
        if(col=='Date'):
            continue;
        df[col] = np.array([float(x) for x in df[col]])

    return df

def random_sample(ticker_list_file,  column = '證券代碼', times = 10):
    

    ticker_list_df = pd.read_csv(ticker_list_file, encoding='utf8', usecols=[column])
    ticker_list = ticker_list_df[column].to_list()
    
    sample = np.random.choice(ticker_list, times)
    sample_list = [i.replace(' ', '') for i in sample.tolist()]

    return sample_list

def file_list_with_directory(file_list, path):
    
    files = [path+i+'.csv' for i in file_list]
    return files

def get_dataframe(data_file, days=0):
    ticker = pd.read_csv(data_file)

    # change the name of columns
    ticker.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

    # set the column "Date" to index
    ticker = ticker.set_index('Date')

    # set index column("Date") from string to datetime.datetime
    ticker.index = pd.to_datetime(ticker.index)

    # sort index column("Date") chronologically
    ticker = ticker.sort_index()
    
    if(days):
        ticker = ticker.tail(days)
    return ticker

## 2. 指標與交易策略

In [3]:
# Define constants
RSI_n = 6
upperBound = 90
lowerBound = 10
middle_line = 50

def RSI_indicator(ticker):




    # Create a empty dataFrame with original index
    # To store some temporary data
    tmp = pd.DataFrame(index=ticker.index)

    # Copy a dataFrame to store result
    # deep=True means make a copy instead of link
    data = ticker.copy(deep=True)

    tmp['rise'] = ticker['Close'] - ticker['Open']

    tmp['rise_only'] = tmp['rise']
    tmp['drop_only'] = tmp['rise']

    tmp['rise_only'].loc[tmp['rise'] < 0] = 0
    tmp['drop_only'].loc[tmp['rise'] > 0] = 0


    tmp['rise_avg'] = tmp['rise_only'].rolling(RSI_n, min_periods=1).mean()
    tmp['drop_avg'] = tmp['drop_only'].rolling(RSI_n, min_periods=1).mean().abs()


    data['RSI'] = tmp['rise_avg'] / (tmp['rise_avg'] + tmp['drop_avg']) * 100
    
    return data

In [8]:
import mplfinance as mpf

def bound(ticker):
    

    upperbound = 95
    lowerbound = 15

    # skip first RSI_n days
    for i in range(RSI_n, data.shape[0]):

        if(data['RSI'][i] < lowerbound):


            data.loc[(data.index[i], 'buy')] = 1
            # print('buy  |', data.index[i], data['Close'][i])
        if(data['RSI'][i] > upperbound):

            data.loc[(data.index[i], 'sell')] = 1
            # print('sell |', data.index[i], data['Close'][i])
            

    return data

def stagnant(ticker):
   
    middle_line = 50
    trading_interval = 6
    trading_cooldown = trading_interval
    stagnant_upper, stagnant_lower = 0, 0


    # skip first RSI_n days
    for i in range(RSI_n, data.shape[0]):

        trading_cooldown += 1
        # 在 upperbound 以上的話， counter就加一
        # 否則counter歸零
        if(data['RSI'][i] >= middle_line):
            stagnant_upper += 1
        else:
            stagnant_upper = 0

        # 在 lowerbound 以下的話， counter就加一
        # 否則counter歸零
        if(data['RSI'][i] <= middle_line ):
            stagnant_lower += 1
        else:
            stagnant_lower = 0

        if(stagnant_upper > trading_interval and trading_cooldown > trading_interval):
            trading_cooldown = 0
            data.loc[(data.index[i], 'buy')] = 1
            # print('buy  |', data.index[i], data['Close'][i])

        if(stagnant_lower > trading_interval and trading_cooldown > trading_interval):
            trading_cooldown = 0
            data.loc[(data.index[i], 'sell')] = 1
            # print('sell |', data.index[i], data['Close'][i])

    
    return data

## 3. 回測模塊

In [6]:
from termcolor import colored
def backtesting(data):
    # how many shares we hold, and the money balance
    have_shares , balance = 0, 0

    # multiple buy and 1 sell as "a trade"
    # record trading times and how many trade are making money
    make_money, trade_times = 0, 0
    
    if np.count_nonzero(data['buy']) == 0:
        return np.nan, np.nan
    
    # we use geometic average
    # this is products of return rate
    return_rate = 1
    for index, row in data.iterrows():

        if(row['buy'] == 1):
            balance -= row['Close']
            have_shares += 1
            print(colored('\tbuy ', 'green'),'|', colored(index.strftime("%Y-%m-%d"), 'green'), '|', colored(row['Close'], 'green'))

        # sell signal or it reach ends
        elif((data.loc[index]['sell'] == 1 or index == data.index.values[-1])and
        # and we have some shares
              have_shares != 0):
            print(colored('\tsell', 'red'),'|', colored(index.strftime("%Y-%m-%d"), 'red'), '|', colored(row['Close'],'red'), "*" , have_shares)
            

            principal = -balance
            balance += row['Close'] * have_shares

            # to calculate accuracy
            trade_times += 1
            if(balance > 0):
                make_money += 1

            # plus 1 avoiding get too small
            return_rate *= 1+(balance / principal)

            # reset balance & shares
            balance = 0
            have_shares = 0


    return_rate = (return_rate-1) * 100.0
    
    accuracy = 0.0
    if(trade_times):
        accuracy = make_money/ trade_times * 100

    return return_rate, accuracy

## 4. 執行回測並搜集資料

In [11]:
random_choice_num = 10
data_directory = '../../parse_data/data/splited/'
ticker_name_file=  '../../parse_data/data/ticker_name.csv'
TIME_INTERVAL = 600

strategy_list = [bound, stagnant]
dictionary_rr = {}
dictionary_acc = {}

for data_file in file_list_with_directory(random_sample(ticker_list_file = ticker_name_file, times = random_choice_num), data_directory):
    
    slim_file_name = (data_file.split('/')[-1]).split('.')[0]

    print(data_file)
    ticker = get_dataframe(data_file, days=200)
    ticker = RSI_indicator(ticker)
    
    strategy_rr= []
    strategy_acc = []
    for i in range(len(strategy_list)):
        
        data = ticker.copy(deep=True)
        
        # initialize 'buy' & 'sell' column by filling with 0
        data['buy'] = np.zeros(ticker.shape[0])
        data['sell'] = np.zeros(ticker.shape[0])

        data = strategy_list[i](data)
        return_rate, acc = backtesting(data)
        
        if return_rate != np.nan:
            return_rate = (round(return_rate, 1))
            acc = (round(acc, 3))
        
        print('\treturn rate: {:>5.1f}% | acc: {:>5.1f}% | {}'.format(return_rate, acc, colored(strategy_list[i].__name__,  'red', attrs=['bold'])))
        strategy_rr.append((return_rate))
        strategy_acc.append((acc))
        print()
    

    #print(strategy_rr)
    #print(strategy_acc)
    
    dictionary_rr[slim_file_name] = strategy_rr
    dictionary_acc[slim_file_name] = strategy_acc



../../parse_data/data/splited/6781.csv
	return rate:   nan% | acc:   nan% | bound

	return rate:   nan% | acc:   nan% | stagnant

../../parse_data/data/splited/3426.csv
	buy  | 2020-06-10 | 35.1
	buy  | 2020-06-11 | 35.1
	buy  | 2020-06-12 | 35.2
	buy  | 2020-06-15 | 34.9
	buy  | 2020-06-16 | 35.5
	sell | 2020-07-02 | 39.0 * 5
	buy  | 2020-08-11 | 38.75
	buy  | 2020-08-12 | 38.55
	buy  | 2020-08-13 | 38.45
	sell | 2020-09-02 | 35.7 * 3
	buy  | 2020-12-16 | 39.05
	buy  | 2020-12-17 | 40.5
	buy  | 2020-12-18 | 39.8
	buy  | 2020-12-21 | 39.1
	buy  | 2020-12-22 | 39.0
	sell | 2020-12-29 | 39.0 * 5
	buy  | 2021-02-04 | 38.5
	buy  | 2021-02-05 | 38.9
	buy  | 2021-02-17 | 38.85
	buy  | 2021-02-18 | 38.85
	buy  | 2021-02-19 | 39.0
	buy  | 2021-02-22 | 38.95
	buy  | 2021-02-23 | 38.8
	buy  | 2021-02-24 | 38.5
	buy  | 2021-02-25 | 38.45
	buy  | 2021-02-26 | 38.45
	buy  | 2021-03-10 | 38.55
	sell | 2021-03-23 | 39.6 * 11
	return rate:   3.7% | acc:  50.0% | bound

	buy  | 2020-06-29 | 37.3
	buy  

## 5. 統計結果

In [12]:
import re

ticker_target = '0050'
df_target = readStock_file('../../parse_data/data/splited/{}.csv'.format(ticker_target))
df_target = df_target.tail(TIME_INTERVAL)
print('Compared target', colored(ticker_target, 'magenta'), ':', end=' ')
print(colored(str(round(((df_target['Close'][-1]/df_target['Close'][0])-1) * 100, 3)) +'%' , 'magenta',attrs=['bold']))
print()

extract_pattern = re.compile("(.+)%")

strategy_names = [i.__name__ for i in strategy_list]

dataframe_return_rate = pd.DataFrame.from_dict(dictionary_rr, orient='index',columns = strategy_names)
dataframe_return_rate_avg = dataframe_return_rate.mean(axis = 0, skipna = True).apply(lambda x : round(x, 3))
dataframe_return_rate_avg = (pd.DataFrame(dataframe_return_rate_avg, columns=['AVG']).transpose())
dataframe_return_rate = dataframe_return_rate.append(dataframe_return_rate_avg)

dataframe_acc = pd.DataFrame.from_dict(dictionary_acc ,orient='index',columns = strategy_names)
dataframe_acc_avg = dataframe_acc.mean(axis = 0, skipna = True).apply(lambda x : round(x, 3))
dataframe_acc_avg = (pd.DataFrame(dataframe_acc_avg, columns=['AVG']).transpose())
dataframe_acc = dataframe_acc.append(dataframe_acc_avg)

pattern = re.compile("(.+)%")
def adding_percent(df):

    for col in df.columns:
        df[col] = df[col].apply(lambda x : str(x) + '%')
    return df

def style_return(df):
    style = df.style.apply(lambda x : ['color:green' if ('nan' not in i and eval(pattern.match(i)[1]) > 1) else '' for i in x])
    style = style.apply(lambda x : ['color:red' if ('nan' not in i and -1.0 > eval(pattern.match(i)[1])) else '' for i in x])
    
    return style
def style_win(df):
    style = df.style.apply(lambda x : ['color:red' if ('nan' not in i  and eval(pattern.match(i)[1]) < 50) else '' for i in x])
    style = style.apply(lambda x : ['color:green' if ('nan' not in i and eval(pattern.match(i)[1]) > 50) else '' for i in x])

    return style




def style_avg(style):
    def custom_avg_row(row):

        # Underline + bold the AVG
        colors = ['background-color: \'\'']*len(row.index)
        colors[-1] = 'font-weight: bold;text-decoration: underline;'
        return colors
    
    style = style.apply(custom_avg_row, axis=0)
    
    # change header color
    headers = {
    'selector': 'th:not(.index_name)',
    'props': [('background-color', '#fff9c7')]
    }
    style = style.set_table_styles([headers])
    
    return style
 

from IPython.display import display

dataframe_return_rate = adding_percent(dataframe_return_rate)
adding_percent(dataframe_acc)
print('return rate:')
display(style_avg(style_return(dataframe_return_rate)))

print()
print('accuracy:')
display(style_avg(style_win(dataframe_acc)))



Compared target 0050 : 60.995%

return rate:


,bound,stagnant
6781,nan%,nan%
3426,3.7%,0.3%
1762,25.9%,-4.8%
5324,22.7%,-12.0%
5288,69.1%,-3.0%
23033,14.2%,5.3%
2734,66.5%,26.2%
3034,176.3%,31.6%
2885,12.9%,5.5%
AVG,48.913%,6.137%



accuracy:


,bound,stagnant
6781,nan%,nan%
3426,50.0%,33.333%
1762,50.0%,0.0%
5324,60.0%,25.0%
5288,100.0%,50.0%
23033,66.667%,50.0%
2734,66.667%,66.667%
3034,100.0%,75.0%
2885,100.0%,66.667%
AVG,74.167%,45.833%
